PAGA stuff1

In [1]:
## load 
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import os 
import csv
from astropy.io import ascii
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import DBSCAN
import warnings
from pathlib import Path
from scipy import stats

In [7]:
code_dir = os.chdir('/mnt/d/Huitian_Practice/exp391')

In [5]:

wk_dir = code_dir + "/1_Scanpy"
Path(wk_dir).mkdir(parents=True, exist_ok=True)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [5]:
analysis_name = ['0', 'basic']
new_wk_dir = wk_dir + "/%s_Scanpy_out_%s"%(analysis_name[0],analysis_name[1])
Path(new_wk_dir).mkdir(parents=True, exist_ok=True)
sp_name='Sc_D3'
print(sp_name, analysis_name)

Sc_D3 ['0', 'basic']


In [6]:
min_gene_nu, min_cell_nu = 500, 3
min_g_nav, max_g_nav = 500,2500
min_m_nav,max_m_nav = 0.0, 0.07
min_c_nav, max_c_nav = 2000, 10000

min_g_a, max_g_a = 500, 7500
min_m_a, max_m_a = 0.00, 0.07
min_c_a, max_c_a = 2000, 60000

fa_n_denoise, fa_pc_denoise = 200, 50
fa_n, fa_pc = 100, 30
max_n_per_cond = 2500


In [10]:
code_dir='/mnt/d/Huitian_Practice'

In [11]:
code_dir = os.getcwd()
mtx_dir = code_dir + "/filtered_feature_bc_matrix"

In [12]:
adata = sc.read_10x_mtx(mtx_dir,var_names='gene_symbols',cache=True)

In [13]:
adata

AnnData object with n_obs × n_vars = 9762 × 31253
    var: 'gene_ids', 'feature_types'

In [15]:
adata.X.astype('float64')

<9762x31253 sparse matrix of type '<class 'numpy.float64'>'
	with 25437127 stored elements in Compressed Sparse Row format>

In [16]:
adata.var_names_make_unique()

In [17]:
sc.pp.filter_cells(adata,min_genes=min_gene_nu)

In [18]:
adata

AnnData object with n_obs × n_vars = 9172 × 31253
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types'

In [19]:
sc.pp.filter_genes(adata,min_cells=min_cell_nu)

In [20]:
adata

AnnData object with n_obs × n_vars = 9172 × 15737
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'n_cells'

In [21]:
mito_genes=adata.var_names.str.startswith('mt-')

In [37]:
adata.obs['percent_mito'] = np.sum(adata[:,mito_genes].X,axis=1).A1/np.sum(adata.X,axis=1).A1
adata.obs['n_counts'] = adata.X.sum(axis=1).A1


In [39]:
adata.obs['n_counts'][0:4]

AAACCCAAGATAGCAT-1    66272.0
AAACCCAGTCCTTTGC-1     6462.0
AAACCCAGTGATTAGA-1    21132.0
AAACCCAGTTCTCTAT-1     6736.0
Name: n_counts, dtype: float32

In [42]:
use_list = []
for i in range(len(adata.obs)):
    if adata.obs['cell_type'][i] != "NAV":
        use_list.append(True)
    else:
        i_genes = ((adata.obs['n_genes'][i] >= min_g_nav) and (adata.obs['n_genes'][i] <= max_g_nav))
        i_mito = ((adata.obs['percent_mito'][i] >= min_m_nav) and (adata.obs['percent_mito'][i] <= max_m_nav))
        i_c = ((adata.obs['n_counts'][i] >= min_c_nav) and (adata.obs['n_counts'][i] <= max_c_nav))
        if all([i_genes,i_mito,i_c]):
            use_list.append(True)
        else:
            use_list.append(False)

KeyError: 'cell_type'